In [ ]:
# %load 10_2022_load_config.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from pathlib import Path
import seaborn as sns
import sys
import plotly.express as px
import plotly.io as pio
import yaml

sns.set_context("notebook", font_scale=1.4)
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 100)
plt.rcParams["figure.figsize"] = (16, 12)
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['figure.autolayout'] = False
plt.rcParams['axes.labelsize'] = 18
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['font.size'] = 16
plt.rcParams['lines.linewidth'] = 2.0
plt.rcParams['lines.markersize'] = 8
plt.rcParams['legend.fontsize'] = 14
pd.set_option('display.float_format', lambda x: '{:,.4f}'.format(x))


config_file = "10_2022_analysis.yaml"
with open(config_file) as file:
    # The FullLoader parameter handles the conversion from YAML
    # scalar values to Python the dictionary format
    configs = yaml.load(file, Loader=yaml.FullLoader)
    
# Run on server:
root = Path(configs['root'])
scratchDir = root/configs['scratchDir']
figuresDir = root/configs['figuresDir']
libraries = configs['libraries']


alphabetClrs = px.colors.qualitative.Alphabet
clrs = ["#f7ba65", "#bf4713", "#9c002f", "#d73d00", "#008080", "#004c4c"]
colors = {'grey': alphabetClrs[8], 
        'light_yellow': clrs[0],
        'darko': clrs[1],
        'maroon':clrs[2],
        'brighto': clrs[3],
        'teal':clrs[4],
        'darkteal':clrs[5]
       }

In [ ]:
libraries

In [ ]:
# for each library make sure the columns in the merged count files == sampleIDs in the sample data file

In [ ]:
countsDir = root/configs['countsDir']
#sampleDir = root/configs['sampleDataDir']
countFiles = [f for f in countsDir.glob("*merged_counts.csv")]

In [ ]:
fsdf = pd.read_csv(root/configs['sampleDataFile'])

In [ ]:
def check_count_merge(cnt_files, fsdf):
    for cnt_file in cnt_files:
        name = cnt_file.stem.split('_mbarq')[0]
        cdf = pd.read_csv(cnt_file, index_col=[0,1]).columns.to_list()
        sdf = fsdf[fsdf.library == name].sampleID.unique()
        all_samples = all([c in cdf for c in sdf])
        print(f"{name}: {all_samples}")
    print('All Done')

In [ ]:
check_count_merge(countFiles, fsdf)

In [ ]:
for c in cdf.columns:
    if c not in sdf.sampleID.unique():
        print(c)

In [ ]:
sdf.shape

In [ ]:
sample_dict = {}
for f in countFiles:
    
    columns = 
    samples = 
    sample_dict[name] = (columns, samples)
    

In [ ]:
c = sample_dict['library_10_2'][0]
d = sample_dict['library_10_2'][1]


In [ ]:
c[3]

In [ ]:
[i in d for i in c]